<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# 6. Modelado de tópicos (libreta de curso)

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



#### Julio Waissman Vilanova (julio.waissman@unison.mx)




Los métodos de modelado de tópicos funcionan de una manera diferentea los vectores de palabras. Para el modelado de tópicos vamos a asumir que tenemos un conjunto de palabras $W = \{w_1, \ldots, w_n\}$ las cuales se pueden indexar (de modo que $w_i$ le corresponde el índice $i$. Por otra parte tenemos una serie de documentos $D = \{d_1, \ldots, d_m\}$ donde cada documento está representado por una secuencia de $n_j$ palabras $d_j = (w^1, \ldots, w^{m_j})$.

Si se asume que tenemos $k$ tópicos, donde $k$ se establece *a priori*, la idea del modelado de tópicos es encontrar dos matrices: la *matriz palabra a tópico* (WTM) $\Phi$ de dimensión $n \times k$ y la *matriz tópico a documento* (TDM) $\Theta$ $k \times m$. Las matrices se forman de tal manera que la entrada $\phi_{i,k}$ de $\Phi$ represente la probabilidad que la palabra $w_i$ conociendo el tópico, mientras que la entrada $\theta_{k, j}$ representa la probabilidad que el documento $d_j$ se trate sobre el tópico $k$.

Aqui lo interesante es que no establecemos los tópicos *a priori*, si no que las matrices se forman al minimizar una funcion objetivo. Dependiendo de la función objetivo la formación de tópicos es diferente, y los métodos de obtención de $\Phi$ y $\Theta$ (que para nosotros van a ser transparentes) también varian de forma importante. Los dos criterios más utilizados son los que dan a lugar a los métodos de [PLSA](https://papers.nips.cc/paper/1654-learning-the-similarity-of-documents-an-information-geometric-approach-to-document-retrieval-and-categorization.pdf) y una [LDA](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), los cuales vimos en términos generales en el curso.

En particula $\Phi$ no solamente provée una especie de matriz de vectores de palabras, si no que, ademas, nos permite ver como están distribuidos los tópicos. Un tópico es definido por el conjunto de palabras con una probabilidad significativa de encontrarse en un documento, si el documento trata sobre el tópico en cuestión. 

## 6.1. Modelado de tópicos con LSA y LDA utilizando `Gensim` 

Para realizar el modelado de tópicos es necesario contar con una bolsa de palabras (de preferencia utilizando la cuenta de las palabras y no únicamente el indicador) por lo que es necesario obtener, normalizar y limpiar el *corpus*. Es importante utilizar el método de BOW que provée *gensim* para realizar el modelado de tópicos.

Vamos a procesar tópicos a partir de los datos de *wikipedia* sobre *políticos argentinos* que obtuvimos y tratamos  en la [primer libreta](./1- Obtencion de textos en espanyol.ipynb). Pero primero veamos algunos para ver que hay que tratar del texto.

In [ ]:
import warnings; warnings.simplefilter('ignore')

import pandas as pd
import re
from random import randint

archivo_pkl = 'datos/wikipedia-politicos-argentina.pkl'
wiki_df = pd.read_pickle(archivo_pkl)

In [ ]:
i = randint(0, wiki_df.shape[0])
print(wiki_df.loc[i,'contenido'])

Y ahora vamos a normalizar el texto. Para esto, y con el fin de encontrar palabras significantes en el análisis de tópicos, no solo hay que eliminar las palabras vacías, si no las palabras que no tienen un valor importante en el discurso. Es por eso que vamos a utilizar el método de etiquetado de parte del discurso para únicamente quedarnos con las palabras que pueden ser más significativas. De nuevo, esta selección depende de lo que queramos extraer. En principio, esto pueden ser pronombres personales (`PROPN`), sustantivos (`NOUN`), adjetivos (`ADJ`) y/o verbos (`VERB`). 

Para esto vamos a utilizar el modulo de *spacy*, pero con el fin de hacer el tratamiento más rápido, vmos a eliminar de la serie de pasos que realiza automáticamente *spacy* la identificación de dependencias y el reconocimiento de entidades.

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')
nlp.disable_pipes('parser', 'ner')
nlp.pipeline

In [ ]:
def normaliza_texto(texto):    
    return [token.lemma_ for token in nlp(texto) 
            if (token.is_alpha and 
                not token.is_stop and
                token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ'])]    

documentos = [normaliza_texto(documento) 
              for documento in wiki_df['contenido'].values]

print(documentos[i])

Ahora tenemos que hacer cuatro pasos importantes: extraer el diccionario a partir del corpus de entrenamiento; convertir los documentos a una representación de BOW; escoger el número de tópicos que queremos analizar; y por último entrenar nuestro modelo (ya sea PLSA, LDA u otro). Pero primero vamos a sacar un documento del corpus para fines de estimación *a posteriori*.

In [ ]:
# Vamos a sacar un documento del corpus
ind_ejemplo = randint(0, len(documentos) - 1)
ejemplo = documentos.pop(ind_ejemplo)

In [ ]:
from gensim import models, corpora

# Número de tópicos
n_topicos = 20

# Genera el diccionario de palabras
diccionario = corpora.Dictionary(documentos)
diccionario.filter_extremes(no_below=5, no_above=0.3)
 
# Extrae las características en forma de BOW
corpus = [diccionario.doc2bow(doc) for doc in documentos]
    
# Genera el modelo LDA
modelo_lda = models.LdaModel(corpus=corpus, num_topics=n_topicos, id2word=diccionario, iterations=100, passes=10)


Y ahor podemos ver algunas cosas que nos permitan analizar nuestro corpus, en primer lugar, como se define a grandes razgos cada tópico

In [ ]:
for (i, topico) in modelo_lda.print_topics(num_topics=n_topicos, num_words=5):
    print(10 * "-" + "topico {}".format(i) + 20 * "-")
    print(topico)

Y ahora vamos a ver como se clasificaría en los diferentes tópicos el documento que no utilizamos:

In [ ]:
texto_ejemplo = wiki_df.loc[ind_ejemplo, 'contenido']
print("El artículo \n<<{}>>\n\n tiene los siguientes tópicos:".format(texto_ejemplo))

topicos_ejemplo = modelo_lda[diccionario.doc2bow(ejemplo)]
topicos_ejemplo.sort(key=lambda x:x[1], reverse=True)
for (topico, peso) in topicos_ejemplo:
    print("\tTopico {} con un peso de {}".format(topico, peso))


Una manera de revisar la calidad del modelo es a partir de la perplejidad de uno o varios documentos

In [ ]:
from math import pow

P = modelo_lda.log_perplexity(corpus)
pow(2, P)

Otro método es analizando la [coherencia de los tópicos](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf), ponemos algunos resultados:

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=modelo_lda, texts=documentos, coherence='c_v')

print("Coherencia del modelo (C_V): {}".format(cm.get_coherence()))
for (topico, coherencia) in enumerate(cm.get_coherence_per_topic()):
    print("\tTópico {}, coherencia {}".format(topico, coherencia))

Y tambien lo podemos utilizar para analizar los tópicos revisando todas las palabras que los definen

In [ ]:
print("Las palabra que definen los tópicos\n")
t_palabras = cm.top_topics_as_word_lists(model=modelo_lda, dictionary=diccionario)
for (topico, palabras) in enumerate(t_palabras):
    print("{:2}-\t{}\n".format(topico, ', '.join(palabras)))

## 6.2 Visualización de tópicos con `pyLDAvis`

Como vimos, explorar los modelos creados y ajustarlos para tratar de extraer mejores tópicos es una tarea ingrata si no se cuenta con herramientas de visualización. El móduo `pyLDAvis` es un clon en python de la misma herramienta (`LDAvis`) que existe en *R*. La gran ventaja es que la herramienta está diseñada para jugar bien con *jupyter*.

`LDAvis` funciona únicamente con modelos que proveen el método `inference`, como LDA que la estimación de tópicos se basa en una inferencia estadística.

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(corpus=corpus, dictionary=diccionario, topic_model=modelo_lda)
panel